<a href="https://colab.research.google.com/github/JeonCollin/mask-detection/blob/main/mask_detection_real_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **0. Mask augmentation**

*   mount google drive
*   MaskTheFace github (https://github.com/aqeelanwar/MaskTheFace.git)
*   include mask augmented face image samples in report



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/content/drive/MyDrive/detect_mask'

In [ ]:
#!unzip data.zip # only at first

In [ ]:
! git clone https://github.com/aqeelanwar/MaskTheFace.git

In [ ]:
! pip install scikit-learn
! pip install wandb
! pip install dlib
! pip install face-recognition
! pip install face-recognition-models
! pip install dotmap

In [ ]:
cd 'MaskTheFace'

In [ ]:
# print current directory
import os
os.getcwd()

In [ ]:
#! python mask_the_face.py --path "/content/drive/MyDrive/detect_mask/data/train/not_wearing_mask" --mask_type "random" #taking mask to raw train image, only at first

In [ ]:
#! python mask_the_face.py --path "/content/drive/MyDrive/detect_mask/data/val/not_wearing_mask" --mask_type "random" #taking mask to raw val image, only at first

# **1. Prepare Data for Training**

*   data_loader using *torchvision.datasets.ImageFolder* for Custom dataset
*   **image augmentation** in *transforms*
*   include augmented face image samples in report



In [ ]:
# import packages
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [ ]:
# Define transformations for data augmentation
data_transform = {
    "train" : transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(112, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=(0.2, 1.5), contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    ]),
    "val" : transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(112),
    transforms.ToTensor(),
    ])}



In [ ]:
train_path = "/content/drive/MyDrive/detect_mask/data/train"
val_path = "/content/drive/MyDrive/detect_mask/data/val"

# write ImageFolder code below // make label to 0 / 1 == nomask/mask
train_data = torchvision.datasets.ImageFolder(root=train_path, transform=data_transform["train"])
val_data = torchvision.datasets.ImageFolder(root=val_path, transform=data_transform["val"])

# check the label
print(train_data.class_to_idx)
print(val_data.class_to_idx)


# **2. Prepare Model**

*   Pytorch ResNet - *ref*. https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py
*   use **ResNet50** from torchvision.model_zoo
*   explore more models in https://pytorch.org/vision/stable/models.html
*   **change the dimension of the classifier**

In [ ]:
# assign device cpu or gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# import packages for importing models
import torchvision.models as models
import torch.nn as nn

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 512),       # 중간에 512차원의 레이어 추가
    nn.BatchNorm1d(512),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(512, 1),
    nn.Sigmoid()
)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 129MB/s]


# **3. Training**


*   write **training code** including belows:
   - hyper parameters such as batch size, learning rate, epoch
   - criterion(loss function such as BCELoss), optimizer(eg. Adam, SGD, etc.)  and scheduler
   - save model weight

*   **print training/validation loss and accuracy** per epoch or iteration
*   inlcude visualizer, **tensorboard**, to show training/validation accuracy and loss


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import wandb
from sklearn import metrics

wandb.login()
# use wandb.init
wandb.init(project="mask-detection")


In [ ]:
import os
import datetime

save_path = "/content/drive/MyDrive/detect_mask/result" +  datetime.datetime.now().strftime("%Y%m%d-%H") + "/"
os.makedirs(save_path, exist_ok=True)

In [ ]:
import torch

# Disable cuDNN
torch.backends.cudnn.enabled = False

# Enable deterministic mode for cuDNN
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []

            for inputs, labels in (train_loader if phase == 'train' else val_loader):
                inputs = inputs.to(device)
                labels = labels.to(device).float()



                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs).squeeze()
                    preds = torch.sigmoid(outputs)>0.5
                    loss = criterion(outputs, labels)
                    optimizer.zero_grad()

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()


                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(torch.sigmoid(outputs).detach().cpu().numpy())

            if phase == 'train':
              scheduler.step()

            epoch_loss = running_loss / len(train_loader.dataset if phase == 'train' else val_loader.dataset)
            epoch_acc = running_corrects.double() / len(train_loader.dataset if phase == 'train' else val_loader.dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # ROC curve, AUC
            if phase == 'val':
                fpr, tpr, _ = metrics.roc_curve(all_labels, all_preds, pos_label=1)
                auc = metrics.auc(fpr, tpr)
                wandb.log({"val_loss": epoch_loss, "val_accuracy": epoch_acc, "val_auc": auc})
                print(f'Val AUC: {auc:.4f}')
            else:
                wandb.log({"train_loss": epoch_loss, "train_accuracy": epoch_acc})

    return model

In [ ]:
# training and validation code can be writed in one function. It's your taste!

In [ ]:
# hyper parameters
num_epochs = 50
learning_rate = 0.01
batch_size = 64

In [ ]:
# optimizer, loss, scheduler
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [ ]:
# data_loader
train_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size, shuffle=True, num_workers=2)


In [ ]:
# ROC curve, AUC (Hint: use sklearn or wandb function, using sklearn to extract fpr, tpr will be bonus score)


In [ ]:
# perform training and validation
model = train_model(model, criterion, optimizer, scheduler,num_epochs=num_epochs)


#save model
torch.save(model.state_dict(), os.path.join(save_path, 'model.pth'))